In [2]:
import numpy as np
import sys
from astropy.io import fits
import matplotlib.pyplot as plt
import sklearn as skl
import pandas as pd
import glob
import gzip
import os
from scipy.interpolate import interp1d
from scipy.signal import fftconvolve
from PyAstronomy.pyasl import rotBroad
from math import sin, pi
from scipy.special import erf                               # Error function
from scipy.signal import fftconvolve 
from lmfit import Parameters, minimize


WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


### testing broadening kernel

In [3]:
def read_HERMES(infile):
    #print("%s: Input file is a HERMES file." % infile)
    header = fits.getheader(infile)

    #bjd = header['MJD-OBS']
    # for files with standard wavelegth array
    if ((header['CTYPE1'] == 'WAVELENGTH') or (header['CTYPE1'] == 'AWAV')):
        flux = fits.getdata(infile, byteorder='little')
        crval = header['CRVAL1']
        cdelt = header['CDELT1']
        naxis1 = header['NAXIS1']
        wave = crval + np.arange(0, naxis1) * cdelt

    # for files that are given in logarithmic wl array
    if (header['CTYPE1'] == 'log(wavelength)'):
        flux = fits.getdata(infile, byteorder='little')
        crval = header['CRVAL1']
        cdelt = header['CDELT1']
        naxis1 = header['NAXIS1']
        wave = np.exp(crval + np.arange(0, naxis1)*cdelt)
    else:
        print("Could not read in HERMES fits file - unknown file type.")
        sys.exit()
    return wave, flux

In [10]:
wave, flux = read_HERMES('00943975_HRF_OBJ_ext_CosmicsRemoved_log_merged_cf_norm.fits')


In [4]:
import warnings
class Model_broad:
    def __init__(self, wave, flux):
        self.x = wave
        self.y = flux
#model_broad = Model_broad(wave, flux)


def Broaden(model, vsini, epsilon=0.5, linear=False, findcont=False):
    # Remove NaN values from the flux array and corresponding wavelength values
    non_nan_idx = ~np.isnan(model.y)
    wave = model.x[non_nan_idx]
    flux = model.y[non_nan_idx]
    
    if not linear:
        x = np.logspace(np.log10(wave[0]), np.log10(wave[-1]), len(wave))
    else:
        x = wave
        
    if findcont:
        # Find the continuum
        model.cont = np.ones_like(flux)  # Placeholder for continuum finding
        
    # Make the broadening kernel
    dx = np.log(x[1] / x[0])
    c = 299792458  # Speed of light in m/s
    lim = vsini / c
    if lim < dx:
        warnings.warn("vsini too small ({}). Not broadening!".format(vsini))
        return Model_broad(wave.copy(), flux.copy())  # Create a copy of the Model object
    
    d_logx = np.arange(0.0, lim, dx)
    d_logx = np.concatenate((-d_logx[::-1][:-1], d_logx))
    alpha = 1.0 - (d_logx / lim) ** 2
    B = (1.0 - epsilon) * np.sqrt(alpha) + epsilon * np.pi * alpha / 4.0  # Broadening kernel
    B /= np.sum(B)  # Normalize

    # Do the convolution
    broadened = Model_broad(wave.copy(), flux.copy())  # Create a copy of the Model object
    broadened.y = fftconvolve(flux, B, mode='same')
    
    return broadened

In [12]:
model_broad = Model_broad(wave, flux)
broadened_model1= Broaden(model_broad, 150000) #vsini in m/s
wv = broadened_model1.x
fl = broadened_model1.y

### fitting gaussians with lmfit

In [ ]:
#reading from linelist to fit
def read_line_list(filename):
    line_centers = []
    line_widths = []

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()  #removing whitespaces
            if not line:
                continue  #skipping empty lines
            parts = line.split()
            center = float(parts[0])
            if len(parts) > 1:
                width = float(parts[1])
            else:
                width = 10.0  #default
            line_centers.append(center)
            line_widths.append(width)

    return line_centers, line_widths

#working gaussian
def gauss(x,a,center,sigma, gamma):
  return a*np.exp(-(x-center)**2/(2*sigma**2)) + gamma

#works for emission lines, for absorption line this is inverted => use neg ampl and a gamma shift
#def gauss(wave, a, center, sigma):
#    return a * np.exp(-0.5 * ((wave - center) / sigma) ** 2)


def generate_data(wave, flux, line_centers, line_widths, wavelength_slices):
    interp_func = interp1d(wave, flux, kind='linear')
    wave_slices = []
    flux_slices = []
    for center, width in zip(line_centers, line_widths):
        new_wave = np.linspace(center - width, center + width, wavelength_slices)
        new_flux = interp_func(new_wave)
        wave_slices.append(new_wave)
        flux_slices.append(new_flux)
    return np.concatenate(wave_slices), np.concatenate(flux_slices)

def generate_model(params, line_centers, line_widths, wavelength_slices):
    model_slices = []
    for i, (center, width) in enumerate(zip(line_centers, line_widths)):
        wave = np.linspace(center - width, center + width, wavelength_slices)
        model = gauss(wave, params[f'a{i}'], params[f'center{i}'], params[f'sigma{i}'], params[f'gamma{i}'])
        model_slices.append(model)
    return np.concatenate(model_slices)

def objective(params, wave, flux, line_centers, line_widths, wavelength_slices):
    wave_data, flux_data = generate_data(wave, flux, line_centers, line_widths, wavelength_slices)
    model = generate_model(params, line_centers, line_widths, wavelength_slices)
    return flux_data - model

def fit_lines(wave, flux, line_centers, line_widths, wavelength_slices=1000):
    params = Parameters()
    wave_data, flux_data = generate_data(wave, flux, line_centers, line_widths, wavelength_slices)
    for i, (center, width) in enumerate(zip(line_centers, line_widths)):
        params.add(f'a{i}', value=-1) #ampl of the line
        params.add(f'center{i}', value=center) #line centre, read the first guess from linelist
        params.add(f'sigma{i}', value=width) # sigma of the line, either read from linelist or a default(1) should do
        params.add(f'gamma{i}', value=1) #initial guess for shifting line to norm level


    result = minimize(objective, params=params, args=(wave_data, flux_data, line_centers, line_widths, wavelength_slices))
    return result

In [ ]:
wave, flux = read_HERMES('00943975_HRF_OBJ_ext_CosmicsRemoved_log_merged_cf_norm.fits')
line_centers, line_widths = read_line_list('line_list.txt')
result = fit_lines(wave, flux, line_centers, line_widths)
result

### fitting with rot broadened gaussian

In [4]:
def read_line_list(filename):
    line_centers = []
    line_widths = []

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()  #removing whitespaces
            if not line:
                continue  #skipping empty lines
            parts = line.split()
            center = float(parts[0])
            if len(parts) > 1:
                width = float(parts[1])
            else:
                width = 10.0  #default
            line_centers.append(center)
            line_widths.append(width)

    return line_centers, line_widths

In [5]:
def gauss(x,a,center,R, gamma):
  sigma = sigma = 4471/ (2.0 * R * np.sqrt(2.0 * np.log(2))) 
  return a*np.exp(-(x-center)**2/(2*sigma**2)) + gamma

def generate_data(wave, flux, line_centers, line_widths, wavelength_slices):
    interp_func = interp1d(wave, flux, kind='linear')
    wave_slices = []
    flux_slices = []
    for center, width in zip(line_centers, line_widths):
        new_wave = np.linspace(center - width, center + width, wavelength_slices)
        new_flux = interp_func(new_wave)
        wave_slices.append(new_wave)
        flux_slices.append(new_flux)
    return np.concatenate(wave_slices), np.concatenate(flux_slices)

In [6]:
import warnings
class Model_broad:
    def __init__(self, wave, flux):
        self.x = wave
        self.y = flux


def Broaden(model, vsini, epsilon=0.5, linear=False, findcont=False):
    # Remove NaN values from the flux array and corresponding wavelength values
    non_nan_idx = ~np.isnan(model.y)
    wvl = model.x[non_nan_idx]
    flx = model.y[non_nan_idx]
    
    dwl = wvl[1] - wvl[0]
    binnu = int(np.floor((((vsini/10)/ 299792.458) * max(wvl)) / dwl)) + 1 #adding extra bins for error handling
    #validIndices = np.arange(len(flx)) + binnu => this was used in rotbroad as a user cond ==> this is always on here
    front_fl = np.ones(binnu) * flx[0]
    end_fl = np.ones(binnu) * flx[-1]
    flux = np.concatenate((front_fl, flx, end_fl))

    front_wv = (wvl[0] - (np.arange(binnu) + 1) * dwl)[::-1]
    end_wv = wvl[-1] + (np.arange(binnu) + 1) * dwl
    wave = np.concatenate((front_wv, wvl, end_wv))

    if not linear:
        x = np.logspace(np.log10(wave[0]), np.log10(wave[-1]), len(wave))
    else:
        x = wave
        
    if findcont:
        # Find the continuum
        model.cont = np.ones_like(flux)  # Placeholder for continuum finding
        
    # Make the broadening kernel
    dx = np.log(x[1] / x[0])
    c = 299792458  # Speed of light in m/s
    lim = vsini / c
    if lim < dx:
        warnings.warn("vsini too small ({}). Not broadening!".format(vsini))
        return Model_broad(wave.copy(), flux.copy())  # Create a copy of the Model object
    
    d_logx = np.arange(0.0, lim, dx)
    d_logx = np.concatenate((-d_logx[::-1][:-1], d_logx))
    alpha = 1.0 - (d_logx / lim) ** 2
    B = (1.0 - epsilon) * np.sqrt(alpha) + epsilon * np.pi * alpha / 4.0  # Broadening kernel
    B /= np.sum(B)  # Normalize

    # Do the convolution
    broadened = Model_broad(wave.copy(), flux.copy())  # Create a copy of the Model object
    broadened.y = fftconvolve(flux, B, mode='same')
    
    return broadened


def generate_broaden(params, line_centers, line_widths, wavelength_slices):
    model_slices = []
    for i, (center, width) in enumerate(zip(line_centers, line_widths)):
        wave = np.linspace(center - width, center + width, wavelength_slices)
        instrum = gauss(wave, params[f'a{i}'], params[f'center{i}'], 20000, params[f'gamma{i}']) #resolution is still hardcoded R=20000 change accordingly
        broad = Broaden(Model_broad(wave, instrum), params['vsini'])

        interp = interp1d(broad.x, broad.y, kind= 'linear')
        broad_flux = interp(wave)
        model_slices.append(broad_flux)
        
    return  np.concatenate(model_slices)



In [7]:
def objective(params, wave, flux, line_centers, line_widths, wavelength_slices):
    wave_data, flux_data = generate_data(wave, flux, line_centers, line_widths, wavelength_slices)
    model = generate_broaden(params, line_centers, line_widths, wavelength_slices)
    return flux_data - model

def fit_lines(wave, flux, line_centers, line_widths, wavelength_slices):
    params = Parameters()
    wave_data, flux_data = generate_data(wave, flux, line_centers, line_widths, wavelength_slices)
    for i, (center, width) in enumerate(zip(line_centers, line_widths)):
        params.add(f'a{i}', value=-1)   # Initial guess for amplitude
        params.add(f'center{i}', value=center)  # Initial guess for center
        params.add(f'gamma{i}', value=1)
    params.add('vsini', value=150000, min = 0, max = 500000)

    result = minimize(objective, params=params, args=(wave_data, flux_data, line_centers, line_widths, wavelength_slices))
    return result

In [8]:
wave, flux = read_HERMES('00943975_HRF_OBJ_ext_CosmicsRemoved_log_merged_cf_norm.fits')
line_centers, line_widths = read_line_list('line_list.txt')
result = fit_lines(wave, flux, line_centers, line_widths, wavelength_slices=1000)
result